In [39]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [40]:
#Import dependencies
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# Import Keras Dependencies
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [41]:
 # Import findspark and initialize.
import findspark
findspark.init()

In [42]:
 # Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("weatherclassification").getOrCreate()

In [43]:
# Load in data
from pyspark import SparkFiles
url = 'https://weatherclassification.s3.amazonaws.com/weather_classification_data.csv'
spark.sparkContext.addFile(url)
print(SparkFiles.getRootDirectory())

spark_df = spark.read.csv(SparkFiles.get("weather_classification_data.csv"), sep=",", header=True, inferSchema=True)
spark_df.show()

/tmp/spark-3f0e7635-9d29-41e7-bcef-264923963261/userFiles-d4fe0f29-b1ae-43e1-8e43-ca1dc54e7f96
+-----------+--------+----------+-----------------+-------------+--------------------+--------+------+---------------+--------+------------+
|Temperature|Humidity|Wind Speed|Precipitation (%)|  Cloud Cover|Atmospheric Pressure|UV Index|Season|Visibility (km)|Location|Weather Type|
+-----------+--------+----------+-----------------+-------------+--------------------+--------+------+---------------+--------+------------+
|         14|      73|       9.5|               82|partly cloudy|             1010.82|       2|Winter|            3.5|  inland|       Rainy|
|         39|      96|       8.5|               71|partly cloudy|             1011.43|       7|Spring|           10.0|  inland|      Cloudy|
|         30|      64|       7.0|               16|        clear|             1018.72|       5|Spring|            5.5|mountain|       Sunny|
|         38|      83|       1.5|               82|        

In [44]:
# Convert PySpark DataFrame to pandas DataFrame
weather_df = spark_df.toPandas()
weather_df.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14,73,9.5,82,partly cloudy,1010.82,2,Winter,3.5,inland,Rainy
1,39,96,8.5,71,partly cloudy,1011.43,7,Spring,10.0,inland,Cloudy
2,30,64,7.0,16,clear,1018.72,5,Spring,5.5,mountain,Sunny
3,38,83,1.5,82,clear,1026.25,7,Spring,1.0,coastal,Sunny
4,27,74,17.0,66,overcast,990.67,1,Winter,2.5,mountain,Rainy


In [45]:
#Create weather data remove seasons
weather_data_no_seasons = weather_df.drop('Season', axis=1)

In [46]:
# Convert categorical data to numeric with `StringIndexer`
weather_data_no_seasons = pd.get_dummies(weather_data_no_seasons).astype(int)
weather_data_no_seasons.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km),Cloud Cover_clear,Cloud Cover_cloudy,Cloud Cover_overcast,Cloud Cover_partly cloudy,Location_coastal,Location_inland,Location_mountain,Weather Type_Cloudy,Weather Type_Rainy,Weather Type_Snowy,Weather Type_Sunny
0,14,73,9,82,1010,2,3,0,0,0,1,0,1,0,0,1,0,0
1,39,96,8,71,1011,7,10,0,0,0,1,0,1,0,1,0,0,0
2,30,64,7,16,1018,5,5,1,0,0,0,0,0,1,0,0,0,1
3,38,83,1,82,1026,7,1,1,0,0,0,1,0,0,0,0,0,1
4,27,74,17,66,990,1,2,0,0,1,0,0,0,1,0,1,0,0


In [47]:
# Split our preprocessed data into our features and target arrays
X = weather_data_no_seasons
y = weather_df['Season']

# Use LabelEncoder to convert categorical season labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = 4
y_one_hot = to_categorical(y_encoded, num_classes)

encoded_labels = label_encoder.classes_
encoded_labels = label_encoder.transform(label_encoder.classes_)
label_mapping = dict(zip(encoded_labels, label_encoder.classes_))
print(f"Labels: {label_mapping}")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, random_state=1)

Labels: {0: 'Autumn', 1: 'Spring', 2: 'Summer', 3: 'Winter'}


In [48]:
# Normalize Data
scaler = MinMaxScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [49]:
# Investigate Data Shape
print('Data Shapes')
print(f'Training Data: {X_train.shape}')
print(f'Training Labels: {y_train.shape}')
print(f'Training Data: {X_test.shape}')
print(f'Training Labels: {y_test.shape}')

Data Shapes
Training Data: (9900, 18)
Training Labels: (9900, 4)
Training Data: (3300, 18)
Training Labels: (3300, 4)


In [50]:
# Define the deep learning model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=64, activation="relu", input_dim=18))
nn_model.add(tf.keras.layers.Dense(units=32, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=4, activation="softmax"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


310/310 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.4119 - loss: 1.1954
Epoch 2/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4299 - loss: 1.1002
Epoch 3/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.4365 - loss: 1.0737
Epoch 4/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4335 - loss: 1.0761
Epoch 5/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4394 - loss: 1.0734
Epoch 6/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4481 - loss: 1.0662
Epoch 7/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4355 - loss: 1.0798
Epoch 8/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4420 - loss: 1.0664
Epoch 9/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.4417 - loss: 1.0605
Epoch 10/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4466 - loss: 1.0739
Epoch 11/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.4684 - loss: 1.0507
Epoch 12/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy

In [88]:
# Make predictions using testing data
predictions = nn_model.predict(X_test)
predictions

104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


array([[2.7371192e-01, 2.4978136e-01, 2.4377458e-01, 2.3273209e-01],
       [2.0645270e-01, 2.4358585e-01, 2.1012467e-01, 3.3983666e-01],
       [1.4871265e-13, 1.3823611e-12, 4.4149360e-14, 9.9999994e-01],
       ...,
       [2.1563348e-01, 2.8408533e-01, 2.8297174e-01, 2.1730949e-01],
       [1.4236433e-07, 8.5378201e-07, 8.4336937e-09, 9.9999905e-01],
       [2.8196713e-01, 2.9159454e-01, 3.4164739e-01, 8.4790982e-02]],
      dtype=float32)

In [89]:
# Convert one-hot encoded values to categorical labes
y_test_labels = np.argmax(y_test, axis = 1)

predictions_labels = np.argmax(predictions, axis=1)



# # Convert prediction labels to 1D array
# predictions_labels = predictions_labels.flatten()

print(predictions_labels)
print(y_test_labels)

[0 3 3 ... 1 3 2]
[3 1 3 ... 3 3 1]


In [90]:
confusion_matrix(y_test_labels, predictions_labels)

array([[  57,  184,   79,  261],
       [  79,  210,   97,  297],
       [  69,  182,  110,  266],
       [  86,  197,   85, 1041]])